In [1]:
import os
import json
import time
import requests
import random
import validators
import pickle

from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

True

In [2]:
# Read the search keywords and frontier URLs set
with open('../data/search_keywords.json', 'r') as fp:
    keywords = json.load(fp)

with open('../data/frontier_urls_initial.json', 'r') as fp:
    frontier_urls = json.load(fp)

print("Number of keywords: ", len(keywords))

Number of keywords:  4715


#### Creating frontier URLs list

We now expand the list of frontier URLs using the keywords

In [5]:
headers = {
    "Ocp-Apim-Subscription-Key": os.getenv("BING_API_KEY"),
    "X-Search-Location": "lat:48.31273;long:-9.03201;re:50"
}

def get_search_results(query):

    try:
        if "tübingen" not in query.lower():
            query += " Tübingen language:en"

        params = {"q": query,  'responseFilter': 'Webpages', 'count': 30}
        response = requests.get(os.environ['BING_ENDPOINT'], headers=headers, params=params)
        search_results = response.json()

        # wait for 1 second to avoid rate limiting
        time.sleep(1)
        return search_results['webPages']['value']
    
    except Exception as e:
        print("Error: ", e)
        return []

search_results = get_search_results("fahrschule")   # still need english results!
search_results

[{'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.0',
  'name': 'Fahrschule Armbruster - Tübingen (72070) -Quick and safe access to your ...',
  'url': 'https://www.fahrschule-123.de/en/driving-school/fahrschule-armbruster-2/',
  'isFamilyFriendly': True,
  'displayUrl': 'https://www.fahrschule-123.de/en/driving-school/fahrschule-armbruster-2',
  'snippet': 'Services. Ausbildungsklassen: Klassen: B, BE, B96. Klassen: A, A2, A1, AM. Klassen: C, C1, CE, C1E. Klassen: L, T. Foreign language in theory training: Arabic, Croatian, English, French, German, Greek, Italian, Polish, Portuguese, Romanian, Russian, Spanish, Turkish. Foreign language in theory class: English, German. Courses.',
  'dateLastCrawled': '2024-04-18T12:26:00.0000000Z',
  'cachedPageUrl': 'http://cc.bingj.com/cache.aspx?q=fahrschule+T%C3%BCbingen+language%3aen&d=4733629904736714&mkt=de-DE&setlang=de-DE&w=A4XNASVB6rFom1hMOGFIN6mSRnZzoV7l',
  'language': 'en',
  'isNavigational': False,
  'noCache': False},
 {'id': '

In [6]:
# Use multiprocessing to create 100 parallel processes to fetch search results
# map each keyword to get_search_results function

import multiprocess as mp

pool = mp.Pool(90)
search_results = pool.map(get_search_results, keywords)

Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'


In [7]:
print("Obtrained {} search results".format(len([x for x in search_results if x])))

Obtrained 4704 search results


In [8]:
keyword_results = zip(keywords, search_results)

# Save the search results to a pickle file
with open('../data/search_results.pkl', 'wb') as fp:
    pickle.dump(keyword_results, fp)

In [9]:
# combine the URLs from the search results and the frontier URLs

frontier_urls = set(frontier_urls)
for keyword, results in keyword_results:
    for result in results:
        url = result['url']
        frontier_urls.add(url)


print("Number of URLs in the frontier: ", len(frontier_urls))

Number of URLs in the frontier:  71943


In [10]:
# Write the updated frontier URLs to a pickle file

with open('../data/frontier_urls.pkl', 'wb') as fp:
    pickle.dump(frontier_urls, fp)

In [11]:
# count the number of URLs in result set
all_results = sum(search_results, [])

print("Total number of URLs in search results: ", len(all_results))
print("Number of unique URLs in search results: ", len(set([x['url'] for x in all_results])))

Total number of URLs in search results:  109417
Number of unique URLs in search results:  70926
